In [1]:
import json
import os


In [2]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open(os.path.join("./results", "data_dict.json"), "r") as f:
    res_obj = json.load(f)

In [3]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

dir_name = "far_shah_b1-b3_EOI_split"

with open(os.path.join("./", dir_name, "test/result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [4]:
res_obj[0]

{'file_name': '/mnt/nis_lab_research/data/se_test/0007dd7ff216_1531619331791546_1531619486460952.jpg',
 'height': 1080,
 'width': 1920,
 'annotations': [{'iscrowd': 0,
   'bbox': [804.4768, 173.2269, 1113.2169, 244.052],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1218.7217, 479.8084, 1308.3959, 522.0554],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [293.754, 11.7548, 332.2216, 51.3159],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [321.4331, 209.2276, 490.6205, 239.5062],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1073.5745, 572.0043, 1113.8889, 609.2933],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [900.1957, 571.0109, 941.4464, 610.8967],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1.878, 7.645, 287.2118, 56.7851],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [729.3143, 571.6522, 767.2729, 612.0123],
   'cate

In [5]:
coco_gt_obj["annotations"][0]

{'id': 30144,
 'image_id': 2114,
 'category_id': 1,
 'segmentation': [],
 'bbox': [1872.5691699604743,
  4.268774703557313,
  41.26482213438749,
  36.996047430830046],
 'ignore': 0,
 'iscrowd': 0,
 'area': 1526.635316908565}

In [6]:
# Group By Image

imgs_gt_pred_list = {}

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"][2:]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, x1 + w, y1 + h])
    
    pred_list = []
    
    for res in res_obj:
        fn2 = os.path.basename(res["file_name"])
        if fn1 == fn2:
            for ann2 in res["annotations"]:
                pred_list.append(ann2["bbox"])
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [7]:
keys = list(imgs_gt_pred_list.keys())


In [8]:
def calculate_iou(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [9]:
thold = .50

In [10]:
iou_scores = []
for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    per_img_scores = []
    for i, pred in enumerate(preds): 
        for j, gt in enumerate(gts):
            iou = calculate_iou(pred, gt)
            if iou >= thold:
                per_img_scores.append(iou)
                break
        if iou < thold: 
            per_img_scores.append(iou)
    iou_scores.append([len(gts), len(preds), per_img_scores])

In [11]:
conf_mat = []
for img in iou_scores:
    tp = len([x for x in img[2] if x > thold])
    num_gt = img[0]
    num_pred = img[1]
    fp = num_pred - tp
    fn = num_gt - tp
    if fn < 0:
        fn = 0
    conf_mat.append([tp, fp, fn])
    

In [12]:
iou_scores_flat = []
for img in iou_scores:
    for item in img[-1]:
        iou_scores_flat.append(item)
corr = len([x for x in iou_scores_flat if x > thold])
accuracy = corr/len(iou_scores_flat)
accuracy

ZeroDivisionError: division by zero

In [ ]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

: 

In [ ]:
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print(precision)
print(recall)
print(f1_score)

: 